<a href="https://colab.research.google.com/github/peterbabulik/QuantumAI/blob/main/QAI_2_50q_ZNE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install qiskit qiskit_ibm_runtime -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.6/378.6 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.8 MB/s eta 0:00:00


In [6]:
import numpy as np
import time
from qiskit.circuit import QuantumCircuit, ParameterVector
from qiskit.circuit.library import EfficientSU2, ZZFeatureMap
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator, EstimatorOptions

# --- CONFIGURATION ---
API_KEY = "your api key here"  # Your Key
BACKEND_NAME = "ibm_torino"
NUM_QUBITS = 50

class LargeScaleQuantumAI:
    def __init__(self):
        print(f"Initializing {NUM_QUBITS}-Qubit AI Model...")

        # 1. Setup Service
        self.service = QiskitRuntimeService(
            channel="ibm_quantum_platform",
            token=API_KEY
        )
        self.backend = self.service.backend(BACKEND_NAME)

        # 2. Define the "Neural Network" Structure
        # A. Feature Map: Encodes data into the 50 qubits
        # We use a reduced feature map to fit 50 inputs
        self.feature_map = ZZFeatureMap(feature_dimension=NUM_QUBITS, reps=1, entanglement='linear')

        # B. The Brain (Ansatz): The trainable layers
        # EfficientSU2 is efficient for hardware (less gates, more entanglement)
        self.ansatz = EfficientSU2(num_qubits=NUM_QUBITS, reps=1, entanglement='linear')

        # Combine them: Data Input -> Quantum Processing
        self.circuit = self.feature_map.compose(self.ansatz)

        # 3. Transpile circuit for the specific chip hardware (Torino)
        print("Compiling circuit for physical hardware...")
        pm = generate_preset_pass_manager(backend=self.backend, optimization_level=3)
        self.isa_circuit = pm.run(self.circuit)

        # 4. Define Observable (What we measure)
        # Measuring Z on the first qubit tells us the classification (1 or -1)
        # (Measuring all 50 is possible but complicates the readout for a simple classifier)
        observable = SparsePauliOp(["I" * (NUM_QUBITS - 1) + "Z"]) # Measure Z on qubit 0
        self.isa_observable = observable.apply_layout(self.isa_circuit.layout)

    def predict(self, input_data, weights):
        """
        Runs a forward pass on the Quantum Computer.
        """
        print(f"--- STARTING INFERENCE ON {BACKEND_NAME} ---")

        # 1. Configure Error Mitigation (ZNE)
        # This allows us to see the 'true' quantum state through the noise of 50 qubits
        options = EstimatorOptions()
        options.resilience.zne_mitigation = True  # <--- CRITICAL FOR 50 QUBITS
        options.default_shots = 2048 # Enough shots to get a stable average

        estimator = Estimator(mode=self.backend, options=options)

        # 2. Combine Data and Weights into one parameter list
        # The circuit expects [Input_Data_Params, Weight_Params]
        full_params = np.concatenate([input_data, weights])

        print(f"Input Vector Size: {len(input_data)}")
        print(f"Weight Vector Size: {len(weights)}")
        print("Submitting job...")

        start_t = time.time()

        # 3. Run Job
        job = estimator.run([(self.isa_circuit, [self.isa_observable], [full_params])])
        print(f"Job ID: {job.job_id()}")
        print("Status: In Queue (This is real hardware, please wait...)")

        result = job.result()

        # 4. Extract Result
        # The result represents the neural activation (-1.0 to 1.0)
        expectation_value = result[0].data.evs[0]
        std_error = result[0].data.stds[0]

        print(f"Done in {time.time() - start_t:.2f}s")
        return expectation_value, std_error

if __name__ == "__main__":
    # --- SIMULATE AN AI APPLICATION ---

    # 1. Instantiate the AI
    ai = LargeScaleQuantumAI()

    # 2. Generate Dummy Data
    # Input: 50 random numbers (representing pixels of an image or financial features)
    input_vector = np.random.uniform(0, np.pi, NUM_QUBITS)

    # Weights: Random weights for the ansatz (in a real scenario, these would be trained)
    # The ansatz has parameters. We need to match the count.
    num_weights = ai.ansatz.num_parameters
    weight_vector = np.random.uniform(0, 2*np.pi, num_weights)

    # 3. Run Inference
    prediction, error = ai.predict(input_vector, weight_vector)

    # 4. Display Logic
    print("\n" + "="*40)
    print("      50-QUBIT AI PREDICTION")
    print("="*40)
    print(f"Raw Output (Expectation): {prediction}")
    print(f"Uncertainty (+/-):        {error}")
    print("-" * 40)

    # Interpret the result as a Binary Classification
    # If Expectation > 0 -> Class A, if < 0 -> Class B
    if prediction > 0:
        print("AI Decision: CLASS A (Probability > 50%)")
    else:
        print("AI Decision: CLASS B (Probability > 50%)")
    print("="*40)

qiskit_runtime_service._discover_account:WARNING:2026-01-30 17:10:54,984: Loading account with the given token. A saved account will not be used.


Initializing 50-Qubit AI Model...


qiskit_runtime_service.__init__:WARNING:2026-01-30 17:10:58,477: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2026-01-30 17:10:58,478: Using instance: open-instance, plan: open
/tmp/ipython-input-2823642193.py:28: DeprecationWarning: The class ``qiskit.circuit.library.data_preparation._zz_feature_map.ZZFeatureMap`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the zz_feature_map function as a replacement. Note that this will no longer return a BlueprintCircuit, but just a plain QuantumCircuit.
  self.feature_map = ZZFeatureMap(feature_dimension=NUM_QUBITS, reps

Compiling circuit for physical hardware...
--- STARTING INFERENCE ON ibm_torino ---
Input Vector Size: 50
Weight Vector Size: 200
Submitting job...
Job ID: d5uec94cqoec73dksc0g
Status: In Queue (This is real hardware, please wait...)
Done in 42.48s

      50-QUBIT AI PREDICTION
Raw Output (Expectation): 0.5139566802372237
Uncertainty (+/-):        0.037156814571298456
----------------------------------------
AI Decision: CLASS A (Probability > 50%)
